## Project 4: Career Prospects In Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

from pprint import pprint
import scipy.stats as stats
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(style="ticks", color_codes=True)


In [3]:
jobs = pd.read_csv('/Users/paklun/Desktop/materials-master/projects/project-4/Career_Database.csv')

In [5]:
jobs.head(5)

,Unnamed: 0,Unnamed: 0.1,Links,Region,Company,Title,Employment Type,Seniority,Industry,Salary,Payment,Description,Requirements
0,0,0,https://www.mycareersfuture.sg/job/data-engine...,South,MONEYSMART SINGAPORE PTE. LTD.,Data Engineer,Full Time,Executive,Information Technology,"$5,000to$7,000",Monthly,Roles & Responsibilities\nMission\nAs part of ...,Requirements\nCompetencies\nDegree in Computer...
1,1,1,https://www.mycareersfuture.sg/job/data-scient...,South,PORTCAST PTE. LTD.,Data Scientist,Full Time,Middle Management,Engineering,"$2,200to$6,000",Monthly,"Roles & Responsibilities\nIn this role, you wi...",Requirements\n● Comfortable working with larg...
2,2,2,https://www.mycareersfuture.sg/job/data-visual...,Central,SINGAPORE PRESS HOLDINGS LIMITED,Data Visualisation Designer,Permanent,Junior Executive,Design,"$3,500to$4,500",Monthly,Roles & Responsibilities\nDigital arm of Engli...,Requirements\nPrior experience in a data visua...
3,3,3,https://www.mycareersfuture.sg/job/data-analys...,Central,GRABTAXI HOLDINGS PTE. LTD.,Data Analyst,Full Time,Executive,Information Technology,NaN,NaN,Roles & Responsibilities\nGet to know our Team...,Requirements\nThe must haves:\nA Bachelor's/Ma...
4,4,4,https://www.mycareersfuture.sg/job/data-center...,Central,AMAZON ASIA-PACIFIC RESOURCES PRIVATE LIMITED,Data Center Engineering Project Engineer APAC,Full Time,Professional,"Design, Engineering","$9,000to$12,000",Monthly,Roles & Responsibilities\nThe Data Center Glob...,Requirements\nBasic Qualifications -\nMinimum ...


In [6]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 13 columns):
Unnamed: 0         4200 non-null int64
Unnamed: 0.1       4200 non-null int64
Links              4200 non-null object
Region             4200 non-null object
Company            4184 non-null object
Title              4184 non-null object
Employment Type    4185 non-null object
Seniority          4120 non-null object
Industry           4185 non-null object
Salary             3716 non-null object
Payment            3716 non-null object
Description        4185 non-null object
Requirements       4045 non-null object
dtypes: int64(2), object(11)
memory usage: 426.6+ KB


In [8]:
jobs.describe()

,Unnamed: 0,Unnamed: 0.1
count,4200.000000,4200.000000
mean,2099.500000,2099.500000
std,1212.579894,1212.579894
min,0.000000,0.000000
25%,1049.750000,1049.750000
50%,2099.500000,2099.500000
75%,3149.250000,3149.250000
max,4199.000000,4199.000000


In [10]:
jobs.shape

(4200, 13)

In [11]:
jobs.drop_duplicates(inplace=True)
jobs.shape

(4200, 13)